## Observations and Insights 

In [1]:
# importing libraries
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# saving paths, opening csv files and saving them into dataframes
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# merging the dataframes
merged_study = pd.merge(mouse_metadata, study_results, on = 'Mouse ID', how = 'outer')

merged_study

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [2]:
# cleaning data

#lines below checks for duplicated ID's on mouse metadata, can be commented if test is succesful
# mice_count = merged_study['Mouse ID'].unique()
# len(mice_count)

#looking for duplicates on the dataframe unique mouses should be 249
mouse_count = merged_study[['Mouse ID', 'Timepoint']].value_counts()
mouse_count = pd.DataFrame(mouse_count).reset_index()
mouse_count = mouse_count.rename(columns={0: 'Count'})

#at least 1 duplicated row, need to check for more
#slicing the dataframe to contain only duplicated values
duplicated_rows = mouse_count.loc[(mouse_count['Count'] > 1),:]

duplicated_rows.head(10)

,Mouse ID,Timepoint,Count
0,g989,15,2
1,g989,0,2
2,g989,10,2
3,g989,5,2
4,g989,20,2


In [3]:
# obtaining a series with the Mouse ID of all the ones that are duplicated

dup_ID = duplicated_rows['Mouse ID'].unique()

print("The duplicated mouse(s) ID are: " + str(dup_ID))

The duplicated mouse(s) ID are: ['g989']


In [4]:
# display the duplicated data to double-check rows to delete
dup_data = merged_study.loc[(merged_study['Mouse ID'].isin(dup_ID)), :]

dup_data


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
908,g989,Propriva,Female,21,26,0,45.000000,0
909,g989,Propriva,Female,21,26,0,45.000000,0
910,g989,Propriva,Female,21,26,5,48.786801,0
911,g989,Propriva,Female,21,26,5,47.570392,0
912,g989,Propriva,Female,21,26,10,51.745156,0
913,g989,Propriva,Female,21,26,10,49.880528,0
914,g989,Propriva,Female,21,26,15,51.325852,1
915,g989,Propriva,Female,21,26,15,53.442020,0
916,g989,Propriva,Female,21,26,20,55.326122,1
917,g989,Propriva,Female,21,26,20,54.657650,1


In [5]:
# deleting duplicated data keeping last values
clean_study = merged_study.drop_duplicates(subset = ['Mouse ID', 'Timepoint'], keep='last')

clean_study

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [6]:
# double checking so that I didn't deleted any valid data
print ("The study was done in: " + str(len(clean_study['Mouse ID'].unique())) + " unique mouses")

The study was done in: 249 unique mouses


## Summary Statistics

In [78]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

#statistical summary calculations

#creating a summary statistics table for tumor volume
statistical_summary = pd.DataFrame(clean_study['Tumor Volume (mm3)'].describe())

#obtaining SEM and adding it to the table
sem = statistical_summary.loc['std', 'Tumor Volume (mm3)'] / np.sqrt(statistical_summary.loc['count', 'Tumor Volume (mm3)'])

statistical_summary = statistical_summary.append([{'Tumor Volume (mm3)': sem}], ignore_index= False)
statistical_summary = statistical_summary.rename(index= {0: 'SEM'})

#grouping the dataframe by drug regimen and obtaining statistical summary
stat_summ_grouped = clean_study.groupby('Drug Regimen').describe().reset_index()

statistical_summary


,Tumor Volume (mm3)
count,1888.000000
mean,50.448411
std,8.904752
min,22.050126
25%,45.000000
50%,48.951421
75%,56.324075
max,78.567014
SEM,0.204937


In [ ]:
stat_summ_grouped


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.



In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot



## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
